# ItaData2024
propositional example

In [7]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [8]:
# ds_path = "/datasets/respiratory_Healthy_Pneumonia"
ds_path = "/datasets/respiratory_Healthy_Bronchiectasis"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [9]:
sr = 8000
audioparams = (
    sr = sr,
    nfft = 256,
    nbands = 14,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = true,
)

function afe(x::AbstractVector{Float64}; sr::Int64, nfft::Int64, nbands::Int64, freq_range::Tuple{Int64, Int64}, db_scale::Bool, get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = sr
    norm = true
    speech_detection = false
    # stft module
    nfft = nfft
    win_type = (:hann, :periodic)
    win_length = nfft
    overlap_length = round(Int, nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = nbands
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = freq_range
    # mel spectrogram module
    db_scale = db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        melspec.spec',
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );
end

afe (generic function with 1 method)

### Compute DataFrame of features

In [10]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; audioparams..., get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz->$j\e[0m" for i in 1:audioparams.nbands]...,
        "\e[$(color_code[:cyan])mcntrd->$j\e[0m", "\e[$(color_code[:cyan])mcrest->$j\e[0m",
        "\e[$(color_code[:cyan])mentrp->$j\e[0m", "\e[$(color_code[:cyan])mflatn->$j\e[0m", "\e[$(color_code[:cyan])mflux->$j\e[0m",
        "\e[$(color_code[:cyan])mkurts->$j\e[0m", "\e[$(color_code[:cyan])mrllff->$j\e[0m", "\e[$(color_code[:cyan])mskwns->$j\e[0m",
        "\e[$(color_code[:cyan])mdecrs->$j\e[0m", "\e[$(color_code[:cyan])mslope->$j\e[0m", "\e[$(color_code[:cyan])msprd->$j\e[0m"
    )
    for j in catch9_f
]...)

# cell 5 - Data compression for propositional analysis
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])

audio_feats = [afe(row[:audio]; audioparams...) for row in eachrow(x)]
push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audio_feats])...)

yc = CategoricalArray(y);

### Compute train and test sets

In [11]:
train_ratio = 0.8

train, test = partition(eachindex(yc), train_ratio, shuffle=true)
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (144, 225) - 144
Test set size: (36, 225) - 36


### Train a model

In [12]:
learned_dt_tree = begin
    Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
    model = Tree(max_depth=-1, )
    mach = machine(model, X_train, y_train)
    fit!(mach)
    fitted_params(mach).tree
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


flux->max < 7.489
├─ sprd->std < 155.0
│  ├─ mel14=3539Hz->std < 0.1605
│  │  ├─ cntrd->std < 213.8
│  │  │  ├─ mel11=2411Hz->qnt < 1.878
│  │  │  │  ├─ Healthy (1/1)
│  │  │  │  └─ Bronchiectasis (23/23)
│  │  │  └─ mel1=409Hz->std < 0.4318
│  │  │     ├─ Bronchiectasis (1/1)
│  │  │     └─ Healthy (4/4)
│  │  └─ mel11=2411Hz->3ac < 0.006044
│  │     ├─ cntrd->bsd < 4.5
│  │     │  ├─ Bronchiectasis (3/3)
│  │     │  └─ Healthy (1/1)
│  │     └─ Healthy (21/21)
│  └─ Healthy (45/45)
└─ mel13=3124Hz->std < 0.4548
   ├─ Bronchiectasis (43/43)
   └─ Healthy (2/2)


### Model inspection & rule study

In [13]:
sole_dt = solemodel(learned_dt_tree)
# Make test instances flow into the model, so that test metrics can, then, be computed.
apply!(sole_dt, X_test, y_test);
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ flux->max < 7.489194711642208
├✔ sprd->std < 155.0391286023021
│├✔ mel14=3539Hz->std < 0.16051373796176915
││├✔ cntrd->std < 213.75397278771902
│││├✔ mel11=2411Hz->qnt < 1.8781197176888846
││││├✔ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││││└✘ Bronchiectasis : (ninstances = 3, ncovered = 3, confidence = 1.0, lift = 1.0)
│││└✘ mel1=409Hz->std < 0.431783542045977
│││ ├✔ Bronchiectasis : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│││ └✘ Healthy : (ninstances = 2, ncovered = 2, confidence = 1.0, lift = 1.0)
││└✘ mel11=2411Hz->3ac < 0.006044238683127572
││ ├✔ cntrd->bsd < 4.5
││ │├✔ Bronchiectasis : (ninstances = 3, ncovered = 3, confidence = 0.67, lift = 1.0)
││ │└✘ Healthy : (ninstances = 2, ncovered = 2, confidence = 1.0, lift = 1.0)
││ └✘ Healthy : (ninstances = 3, ncovered = 3, confidence = 1.0, lift = 1.0)
│└✘ Healthy : (ninstances = 8, ncovered = 8, confidence = 0.75, lift = 1.0)
└✘ mel13=3124Hz->std < 0.4547766847788547
 ├✔ Bronchiec

### Extract rules that are at least as good as a random baseline model

In [14]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (mel14=3539Hz->std < 0.16051373796176915) ∧ (cntrd->std < 213.75397278771902) ∧ (¬(mel11=2411Hz->qnt < 1.8781197176888846))  ↣  Bronchiectasis : (ninstances = 36, ncovered = 3, coverage = 0.08, confidence = 1.0, lift = 1.8, natoms = 5)
▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (mel14=3539Hz->std < 0.16051373796176915) ∧ (¬(cntrd->std < 213.75397278771902)) ∧ (¬(mel1=409Hz->std < 0.431783542045977))  ↣  Healthy : (ninstances = 36, ncovered = 2, coverage = 0.06, confidence = 1.0, lift = 2.25, natoms = 5)
▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (¬(mel14=3539Hz->std < 0.16051373796176915)) ∧ (mel11=2411Hz->3ac < 0.006044238683127572) ∧ (cntrd->bsd < 4.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 3, coverage = 0.08, confidence = 0.67, lift = 1.2, natoms = 5)
▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (¬(mel14=3539Hz->std < 0.16051

### Simplify rules while extracting and prettify result

In [15]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (flux->max < 7.49) ∧ (sprd->std < 155.04) ∧ (mel14=3539Hz->std < 0.16) ∧ (cntrd->std < 213.75) ∧ (mel11=2411Hz->qnt ≥ 1.88)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 3, coverage = 0.08, confidence = 1.0, lift = 1.8, natoms = 5)
▣ (flux->max < 7.49) ∧ (sprd->std < 155.04) ∧ (mel14=3539Hz->std < 0.16) ∧ (cntrd->std ≥ 213.75) ∧ (mel1=409Hz->std ≥ 0.43)  ↣  Healthy : (ninstances = 36, ncovered = 2, coverage = 0.06, confidence = 1.0, lift = 2.25, natoms = 5)
▣ (flux->max < 7.49) ∧ (sprd->std < 155.04) ∧ (mel14=3539Hz->std ≥ 0.16) ∧ (mel11=2411Hz->3ac < 0.01) ∧ (cntrd->bsd < 4.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 3, coverage = 0.08, confidence = 0.67, lift = 1.2, natoms = 5)
▣ (flux->max < 7.49) ∧ (sprd->std < 155.04) ∧ (mel14=3539Hz->std ≥ 0.16) ∧ (mel11=2411Hz->3ac < 0.01) ∧ (cntrd->bsd ≥ 4.5)  ↣  Healthy : (ninstances = 36, ncovered = 2, coverage = 0.06, confidence = 1.0, lift = 2.25, natoms = 5)
▣ (flux->max < 7.49) ∧ (sprd->std < 155.04) ∧ (mel14=3539Hz->std ≥ 

### Directly access rule metrics

In [16]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

7-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 36, ncovered = 3, coverage = 0.08333333333333333, confidence = 1.0, lift = 1.7999999999999998, natoms = 5)
 (ninstances = 36, ncovered = 2, coverage = 0.05555555555555555, confidence = 1.0, lift = 2.25, natoms = 5)
 (ninstances = 36, ncovered = 3, coverage = 0.08333333333333333, confidence = 0.6666666666666666, lift = 1.2, natoms = 5)
 (ninstances = 36, ncovered = 2, coverage = 0.05555555555555555, confidence = 1.0, lift = 2.25, natoms = 5)
 (ninstances = 36, ncovered = 3, coverage = 0.08333333333333333, confidence = 1.0, lift = 2.25, natoms = 4)
 (ninstances = 36, ncovered = 8, coverage = 0.2222222222222222, confidence = 0.75, lift = 1.6875, natoms = 2)
 (ninstances = 36, ncovered = 11, coverage = 0.3055555555555556, confidence = 0.9090909090909091, lift = 1.6363636363636362, natoms = 2)

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [17]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (mel14=3539Hz->std < 0.16051373796176915) ∧ (cntrd->std ≥ 213.75397278771902) ∧ (mel1=409Hz->std ≥ 0.431783542045977)  ↣  Healthy : (ninstances = 36, ncovered = 2, coverage = 0.05555555555555555, confidence = 1.0, lift = 2.25, natoms = 5, height = 4)
▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (mel14=3539Hz->std ≥ 0.16051373796176915) ∧ (mel11=2411Hz->3ac < 0.006044238683127572) ∧ (cntrd->bsd ≥ 4.5)  ↣  Healthy : (ninstances = 36, ncovered = 2, coverage = 0.05555555555555555, confidence = 1.0, lift = 2.25, natoms = 5, height = 4)
▣ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (mel14=3539Hz->std ≥ 0.16051373796176915) ∧ (mel11=2411Hz->3ac < 0.006044238683127572) ∧ (cntrd->bsd < 4.5)  ↣  Bronchiectasis : (ninstances = 36, ncovered = 3, coverage = 0.08333333333333333, confidence = 0.6666666666666666, lift = 1.2, natoms = 5, height = 4)
▣ (flux->max < 7.489194711642208) ∧ 

### Pretty table of rules and their metrics

In [18]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────┬────────────┬──────────┬───────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                   Antecedent │     Consequent │ ninstances │ ncovered │  coverage │ confidence │    lift │ natoms │ height │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────────┼────────────┼──────────┼───────────┼────────────┼─────────┼────────┼────────┤
│ (flux->max < 7.489194711642208) ∧ (sprd->std < 155.0391286023021) ∧ (mel14=3539Hz->std < 0.16051373796176915) ∧ (cntrd->std < 213.7539727